# Dataset

In [1]:
with open('hafez_norm.txt', 'r') as file:
    content = file.read()
print(content)


  	
الا یا ایها الساقی ادر کاسا و ناولها
که عشق آسان نمود اول ولی افتاد مشکل ها
به بوی نافه ای کاخر صبا زان طره بگشاید
ز تاب جعد مشکینش چه خون افتاد در دل ها
مرا در منزل جانان چه امن عیش چون هر دم
جرس فریاد می دارد که بربندید محمل ها
به می سجاده رنگین کن گرت پیر مغان گوید
که سالک بی خبر نبود ز راه و رسم منزل ها
شب تاریک و بیم موج و گردابی چنین هایل
کجا دانند حال ما سبکباران ساحل ها
همه کارم ز خود کامی به بدنامی کشید آخر
نهان کی ماند آن رازی کز او سازند محفل ها
حضوری گر همی خواهی از او غایب مشو حافظ
متی ما تلق من تهوی دع الدنیا و اهملها
صلاح کار کجا و من خراب کجا
ببین تفاوت ره کز کجاست تا به کجا
دلم ز صومعه بگرفت و خرقه سالوس
کجاست دیر مغان و شراب ناب کجا
چه نسبت است به رندی صلاح و تقوا را
سماع وعظ کجا نغمه رباب کجا
ز روی دوست دل دشمنان چه دریابد
چراغ مرده کجا شمع آفتاب کجا
چو کحل بینش ما خاک آستان شماست
کجا رویم بفرما از این جناب کجا
مبین به سیب زنخدان که چاه در راه است
کجا همی روی ای دل بدین شتاب کجا
بشد که یاد خوشش باد روزگار وصال
خود آن کرشمه کجا رفت و آن عتاب کجا
قرار و خواب ز حاف

In [2]:
content = content.strip()
print(content)

الا یا ایها الساقی ادر کاسا و ناولها
که عشق آسان نمود اول ولی افتاد مشکل ها
به بوی نافه ای کاخر صبا زان طره بگشاید
ز تاب جعد مشکینش چه خون افتاد در دل ها
مرا در منزل جانان چه امن عیش چون هر دم
جرس فریاد می دارد که بربندید محمل ها
به می سجاده رنگین کن گرت پیر مغان گوید
که سالک بی خبر نبود ز راه و رسم منزل ها
شب تاریک و بیم موج و گردابی چنین هایل
کجا دانند حال ما سبکباران ساحل ها
همه کارم ز خود کامی به بدنامی کشید آخر
نهان کی ماند آن رازی کز او سازند محفل ها
حضوری گر همی خواهی از او غایب مشو حافظ
متی ما تلق من تهوی دع الدنیا و اهملها
صلاح کار کجا و من خراب کجا
ببین تفاوت ره کز کجاست تا به کجا
دلم ز صومعه بگرفت و خرقه سالوس
کجاست دیر مغان و شراب ناب کجا
چه نسبت است به رندی صلاح و تقوا را
سماع وعظ کجا نغمه رباب کجا
ز روی دوست دل دشمنان چه دریابد
چراغ مرده کجا شمع آفتاب کجا
چو کحل بینش ما خاک آستان شماست
کجا رویم بفرما از این جناب کجا
مبین به سیب زنخدان که چاه در راه است
کجا همی روی ای دل بدین شتاب کجا
بشد که یاد خوشش باد روزگار وصال
خود آن کرشمه کجا رفت و آن عتاب کجا
قرار و خواب ز حافظ طمع

In [3]:
# Split the content into lines
lines = content.split('\n')

print(f"Number of lines: {len(lines)}")

Number of lines: 10226


# Tokenizing & Embedding

In [4]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import torch


# Initialize tokenizer with BPE model
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()

# Add more special tokens for poem structure
trainer = BpeTrainer(
    special_tokens=["<unk>", "<pad>", "<bos>", "<eos>", "<versel1>", "<versel2>", "<linebreak>"],
    vocab_size=10000,
    show_progress=True
)

# When preparing your data for training, add the structure markers
structured_lines = []
for i in range(0, len(lines), 2):
    if i+1 < len(lines):
        # Format: <versel1> first_half <versel2> second_half <linebreak>
        structured_line = f"<versel1> {lines[i]} <versel2> {lines[i+1]} <linebreak>"
        structured_lines.append(structured_line)

# Create corpus with structured text
with open("structured_corpus.txt", "w", encoding="utf-8") as f:
    for line in structured_lines:
        f.write(line + "\n")

# Train tokenizer on the structured corpus
tokenizer.train(["structured_corpus.txt"], trainer)

# For inference/generation, you can:
# 1. Start with "<bos> <versel1> your_prompt"
# 2. Let the model generate until it produces "<linebreak>"
# 3. Extract the completed poem line

# When generating, you can guide the model:
def prepare_prompt(first_versel):
    return f"<bos> <versel1> {first_versel}"

# The model should then generate the second versel and end with <linebreak>



# Save the tokenizer for later use
tokenizer.save("persian_poetry_bpe.json")

# Join all structured lines into a single text with appropriate separators
full_text = "".join(structured_lines)

print(full_text)
# Encode the entire text at once
encoding = tokenizer.encode(full_text)

# Get tokens and token IDs
tokens = encoding.tokens
token_ids = encoding.ids

print(f"First 20 tokens: {tokens[:20]}")
print(f"Total number of tokens: {len(tokens)}")


# Now you can use these sequences to train your model
# vocab_size is still the same
vocab_size = tokenizer.get_vocab_size()
print(vocab_size)




<versel1> الا یا ایها الساقی ادر کاسا و ناولها <versel2> که عشق آسان نمود اول ولی افتاد مشکل ها <linebreak><versel1> به بوی نافه ای کاخر صبا زان طره بگشاید <versel2> ز تاب جعد مشکینش چه خون افتاد در دل ها <linebreak><versel1> مرا در منزل جانان چه امن عیش چون هر دم <versel2> جرس فریاد می دارد که بربندید محمل ها <linebreak><versel1> به می سجاده رنگین کن گرت پیر مغان گوید <versel2> که سالک بی خبر نبود ز راه و رسم منزل ها <linebreak><versel1> شب تاریک و بیم موج و گردابی چنین هایل <versel2> کجا دانند حال ما سبکباران ساحل ها <linebreak><versel1> همه کارم ز خود کامی به بدنامی کشید آخر <versel2> نهان کی ماند آن رازی کز او سازند محفل ها <linebreak><versel1> حضوری گر همی خواهی از او غایب مشو حافظ <versel2> متی ما تلق من تهوی دع الدنیا و اهملها <linebreak><versel1> صلاح کار کجا و من خراب کجا <versel2> ببین تفاوت ره کز کجاست تا به کجا <linebreak><versel1> دلم ز صومعه بگرفت و خرقه سالوس <versel2> کجاست دیر مغان و شراب ناب کجا <linebreak><versel1> چه نسبت است به رندی صلاح و تقوا را <versel2> سماع

In [5]:
# Convert to tensor
token_ids_tensor = torch.tensor(token_ids)
print(f"Shape of token_ids tensor: {token_ids_tensor.shape}")

Shape of token_ids tensor: torch.Size([87706])


In [6]:

def generate_x_y_batches(data:torch.tensor, context_len:int , step:int, batch_size:int):
    x_batches = []
    y_batches = []
    
    for i in range(0, len(data)-context_len, step):
        x_batches.append(data[i:i+context_len])
        y_batches.append(data[i+1: i+1+context_len])
    
    x_batches = torch.stack(x_batches)
    y_batches = torch.stack(y_batches)


    num_sequences = len(x_batches)
    num_complete_batches = (num_sequences // batch_size) * batch_size

    x_batches = x_batches[:num_complete_batches]
    y_batches = y_batches[:num_complete_batches]

    x_batch_tensor = x_batches.view(-1, batch_size, context_len)
    y_batch_tensor = y_batches.view(-1, batch_size, context_len)

    return x_batch_tensor, y_batch_tensor




# Model

In [7]:
from torch import nn
from torch.nn import functional as F

class HeadAttention(nn.Module):

    def __init__(self, embedding_dim:int , head_size:int, context_len:int , dropout :float):
        super().__init__()
        self.head_size = head_size
        self.embedding_dim = embedding_dim
        self.query = nn.Linear(self.embedding_dim, self.head_size, bias=False)
        self.key = nn.Linear(self.embedding_dim, self.head_size, bias=False)
        self.value = nn.Linear(self.embedding_dim, self.head_size, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        self.dropput = nn.Dropout(dropout)
        self.register_buffer('tril', torch.tril(torch.ones(context_len, context_len)))
        
    
    
    def forward(self, x):
        # x shape: (batch_size, context_len, embedding_dim)
        # Project input x into query, key, and value
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)

        # Compute attention scores (use scaled dot-product attention)
        # queries shape: (batch_size, context_len, head_size)
        # keys shape: (batch_size, context_len, head_size)
        # Scale dot-product by sqrt(head_size)
        scores = (queries @ keys.transpose(-2, -1)) / (self.head_size ** 0.5)

        scores = scores.masked_fill(self.tril== 0, float('-inf')) # (B, T, T)
        scores = self.softmax(scores) # (B, T, T)
        scores = self.dropput(scores)
        attention = scores @ values  # Apply softmax to get attention weights

        
        return attention



class FeedForward(nn.Module):
    def __init__(self, embedding_dim:int, dropout:float):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = 4*embedding_dim
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(self.embedding_dim, self.hidden_dim)
        self.linear2 = nn.Linear(self.hidden_dim, self.embedding_dim)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return self.dropout(x)



class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim:int, context_len:int,  head_size:int, num_heads:int, dropout :float):
        super().__init__()
        self.head_size = head_size
        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        self.heads = nn.ModuleList([HeadAttention(embedding_dim, head_size, context_len,  dropout) for _ in range(num_heads)])
        self.out = nn.Linear(self.num_heads * self.head_size, self.embedding_dim)

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        heads_ouput = [head(x) for head in self.heads]
        heads_ouput_tensor = torch.stack(heads_ouput, dim=3)
        heads_ouput_tensor = heads_ouput_tensor.view(x.shape[0], x.shape[1], self.num_heads * self.head_size)

        return self.dropout(self.out(heads_ouput_tensor))


class TransformerBlock(nn.Module):
    def __init__(self, embedding_dim, context_len ,  head_size, num_heads, dropout ):
        super().__init__()
        head_size = embedding_dim//num_heads
        self.multi_head_attention = MultiHeadAttention(embedding_dim=embedding_dim, context_len=context_len, head_size=head_size, num_heads=head_size, dropout=dropout)
        self.feed_forward = FeedForward(embedding_dim, dropout)
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)
    
    def forward(self, x):
        attention_output = self.multi_head_attention(x)
        x = self.layer_norm1(x + attention_output)
        feed_forward_output = self.feed_forward(x)
        return self.layer_norm2(x + feed_forward_output)
    


class GPTLanguageModel(nn.Module):
    def __init__(self,vocab_size:int , embedding_dim:int, context_len:int, head_size:int, num_heads:int, num_transformer_blocks:int, dropout:float):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.context_len = context_len
        self.head_size = head_size
        self.num_heads = num_heads
        self.num_transformer_blocks = num_transformer_blocks
        self.dropout = dropout
        self.transformer_blocks = nn.ModuleList([TransformerBlock(embedding_dim, context_len , head_size, num_heads, dropout) for _ in range(num_transformer_blocks)])
        self.layernorm_final = nn.LayerNorm(embedding_dim) # final layer norm
        self.out = nn.Linear(embedding_dim, vocab_size)

        self.token_embedding_table = nn.Embedding(vocab_size, embedding_dim)
        self.position_embedding_table = nn.Embedding(context_len, embedding_dim)

    
    def forward(self, idx):
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(self.context_len, device=idx.device))
        x = tok_emb + pos_emb
        
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x)
        
        x = self.layernorm_final(x) # (B,T,C)
        logits = self.out(x) # (B,T,vocab_size)
        return logits


@torch.no_grad()    
def generate(model, prompt_ids, max_new_tokens, context_len,device='cuda', temperature=1.0):
    """
    prompt_ids: (seq_len, ) - starting token IDs
    max_new_tokens: how many new tokens to generate
    temperature: controls randomness (lower = more deterministic)
    """
    model.eval()
    
    # Get pad token ID
    pad_id = tokenizer.token_to_id("<pad>")
    
    # Process input - pad if needed
    if len(prompt_ids) < context_len:
        padded_prompt = prompt_ids + [pad_id] * (context_len - len(prompt_ids))
    else:
        padded_prompt = prompt_ids[-context_len:]
    
    # Convert to tensor and add batch dimension
    idx = torch.tensor([padded_prompt], dtype=torch.long).to(device)
    
    # Print starting text
    print("\n--- Starting with: ---")
    print(f"{tokenizer.decode(prompt_ids)}")
    print("---------------------------------\n")
    
    
    # Generate tokens sequentially
    for _ in range(max_new_tokens):
        # Crop context if it's getting too long
        idx_cond = idx if idx.size(1) <= context_len else idx[:, -context_len:]
        
        # Forward pass to get logits
        logits = model(idx_cond)
        
        # Focus on the last token's prediction
        logits = logits[:, -1, :] # (B, vocab_size)
        
        # Apply temperature to control randomness
        logits = logits / temperature
        
        # Apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)
        
        # Sample from the distribution
        next_idx = torch.multinomial(probs, num_samples=1) # (B, 1)
        
        # Append the new token to our sequence
        idx = torch.cat((idx, next_idx), dim=1)
        
        # Print the current state
        next_token = tokenizer.decode([next_idx[0].item()], skip_special_tokens=False)
        generated_so_far = tokenizer.decode(idx[0].tolist(), skip_special_tokens=False)
        
        # Check if we've generated a complete poem stanza
        if ("<linebreak>" in next_token) or ("<eos>" in next_token):
            print("\n--- Complete poem generated ---")
            print(generated_so_far)
            print("---------------------------------\n")
            break
        
            
    return idx

    

# Training

In [8]:
from tqdm import tqdm
from torch.nn import functional as F
import wandb



vocab_size = tokenizer.get_vocab_size()
num_train_epochs = 50
batch_size = 512
step_token = 1
context_len = 50
embedding_dim = 100
head_size=8
num_heads = 8
num_transformer_blocks = 5
dropout = 0.2
train_split_percent = 0.75
generate_temperature = 0.8
device = 'cuda' if torch.cuda.is_available() else 'cpu'



# Data 
Xb, yb = generate_x_y_batches(data=token_ids_tensor, context_len = context_len , step =step_token, batch_size = batch_size)

Xb_train, yb_train = Xb[:int(train_split_percent*len(Xb)), :, :], yb[:int(train_split_percent*len(Xb)), :, :]
Xb_test, yb_test = Xb[int(train_split_percent*len(Xb)):, :, :], yb[int(train_split_percent*len(Xb)):, :, :]

# Move to device and ensure gradients are maintained
Xb_train, yb_train = Xb_train.to(device), yb_train.to(device)
Xb_test, yb_test = Xb_test.to(device), yb_test.to(device)

print(f"Xb_train.shape: {Xb_train.shape}")
print(f"Xb_test.shape: {Xb_test.shape}")

# Model
model = GPTLanguageModel(vocab_size=vocab_size, embedding_dim=embedding_dim, context_len=context_len, num_transformer_blocks = num_transformer_blocks , head_size=head_size, num_heads=num_heads, dropout=dropout)
model.to(device)


# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)



# Initialize wandb at the beginning of your script
wandb.init(
    project="persian-poetry-gpt",
    config={
        "vocab_size": vocab_size,
        "num_train_epochs": num_train_epochs,
        "batch_size": batch_size,
        "context_len": context_len,
        "embedding_dim": embedding_dim,
        "head_size": head_size,
        "num_heads": num_heads,
        "num_transformer_blocks": num_transformer_blocks,
        "dropout": dropout,
        "train_split_percent": train_split_percent,
    }
)

# Training loop with wandb logging
for epoch in range(num_train_epochs):
    model.train()
    total_loss = 0
    num_batches = len(Xb_train)
    
    # Progress bar
    progress_bar = tqdm(enumerate(zip(Xb_train, yb_train)), total=num_batches, 
                        desc=f"Epoch {epoch+1}/{num_train_epochs}")
    
    for i, (Xb_mini, yb_mini) in progress_bar:
        optimizer.zero_grad()

        # Forward pass         
        yb_train_prediction = model(Xb_mini)
        
        # Calculate loss 
        loss = F.cross_entropy(yb_train_prediction.view(batch_size*context_len, vocab_size), 
                              yb_mini.view(batch_size*context_len))
        
        loss.backward()
        optimizer.step()
        
        # Update total loss
        total_loss += loss.item()
        
        # Update progress bar
        progress_bar.set_postfix({
            "batch_loss": f"{loss.item():.4f}", 
            "avg_loss": f"{total_loss/(i+1):.4f}"
        })
        
        # Log batch metrics (every 10 batches to avoid flooding)
        if i % 10 == 0:
            wandb.log({
                "batch": i + epoch * num_batches,
                "train_batch_loss": loss.item(),
                "learning_rate": optimizer.param_groups[0]['lr']
            })
    
    # Calculate epoch average loss
    avg_epoch_loss = total_loss / num_batches
    print(f"Epoch {epoch+1}/{num_train_epochs}, Train Loss: {avg_epoch_loss:.4f}")

    # Evaluation
    with torch.no_grad():
        model.eval()
        total_eval_loss = 0
        num_eval_batches = len(Xb_test)
        
        for Xb_mini, yb_mini in tqdm(zip(Xb_test, yb_test), total=num_eval_batches,
                                    desc=f"Evaluating Epoch {epoch+1}"):
            # Forward pass         
            yb_test_prediction = model(Xb_mini)
            
            # Calculate loss 
            loss = F.cross_entropy(yb_test_prediction.view(batch_size*context_len, vocab_size), 
                                  yb_mini.view(batch_size*context_len))
            
            # Update total loss
            total_eval_loss += loss.item()
        
        # Calculate evaluation average loss
        avg_eval_epoch_loss = total_eval_loss / num_eval_batches
        print(f"Epoch {epoch+1}/{num_train_epochs}, Evaluation Loss: {avg_eval_epoch_loss:.4f}")
        
        # Log epoch metrics
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_epoch_loss,
            "val_loss": avg_eval_epoch_loss,
        })
    
        # Generate sample text once per epoch
        prompt_ids = Xb_test[0, 0, :].tolist()
        generated_text = generate(model, prompt_ids=prompt_ids, max_new_tokens=context_len, 
                                 context_len=context_len, device=device, temperature=0.8)

        full_poem = tokenizer.decode(generated_text[0].tolist())
        
        # Log generated text sample
        wandb.log({
            "epoch": epoch + 1,
            "generated_text": wandb.Html(f"<p>{full_poem}</p>")
        })
        
        # Save model checkpoint (either periodically or on best validation loss)
        # if (epoch + 1) % 5 == 0:  # Save every 5 epochs
        #     torch.save(model.state_dict(), f"model_checkpoint_epoch_{epoch+1}.pt")
        #     wandb.save(f"model_checkpoint_epoch_{epoch+1}.pt")

# Finish wandb run
wandb.finish()



Xb_train.shape: torch.Size([128, 512, 50])
Xb_test.shape: torch.Size([43, 512, 50])


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mostafa-alavi25 (mostafa-alavi25-free) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/50:   4%|▍         | 5/128 [00:01<00:29,  4.21it/s, batch_loss=8.6584, avg_loss=8.9307]

Epoch 1/50: 100%|██████████| 128/128 [00:12<00:00, 10.43it/s, batch_loss=6.3797, avg_loss=6.7580]


Epoch 1/50, Train Loss: 6.7580


...

Epoch 50/50: 100%|██████████| 128/128 [00:10<00:00, 12.17it/s, batch_loss=1.5856, avg_loss=1.7019]


Epoch 50/50, Train Loss: 1.7019


Evaluating Epoch 50: 100%|██████████| 43/43 [00:00<00:00, 52.22it/s]


Epoch 50/50, Evaluation Loss: 10.2652

--- Starting with: ---
صبوری ایشان دریده ای از چشم بخت خویش مبادت گزند از آنک در دلبری به غایت خوبی رسیده ای منعم مکن ز عشق وی ای مفتی زمان معذور دارمت که تو او را ندیده ای آن سرزنش که کرد تو را دوست
---------------------------------


--- Complete poem generated ---
صبوری ایشان دریده ای <linebreak> <versel1> از چشم بخت خویش مبادت گزند از آنک <versel2> در دلبری به غایت خوبی رسیده ای <linebreak> <versel1> منعم مکن ز عشق وی ای مفتی زمان <versel2> معذور دارمت که تو او را ندیده ای <linebreak> <versel1> آن سرزنش که کرد تو را دوست به کاشانه دوست <versel2> هر جا که سرانجام کرد افسون و دل به هم <linebreak>
---------------------------------



batch,▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇█████
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_batch_loss,█▇▇▇▇▆▆▆▅▅▄▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▁▁▁▁
train_loss,█▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
batch,6392
epoch,50
learning_rate,0.001
train_batch_loss,1.67042
train_loss,1.7019


In [10]:
torch.save(model.state_dict(), f"model_checkpoint_final.pt")

# Generating

In [11]:
# Get a starting prompt (first half of a versel)
prompt = "<bos> <versel1>  از چشم بخت خویش مبادت گزند"  

# Encode the prompt
prompt_ids = tokenizer.encode(prompt).ids

# Generate poem continuation
generated = generate(model, prompt_ids, max_new_tokens=context_len, context_len=context_len, temperature=0.8, device=device)

# Decode the full generated text
full_poem = tokenizer.decode(generated[0].tolist())
print("Complete generation:")
print(full_poem)


--- Starting with: ---
از چشم بخت خویش مبادت گزند
---------------------------------


--- Complete poem generated ---
<bos> <versel1> از چشم بخت خویش مبادت گزند <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> نعمت فریب <versel2> کوتاه و از می و قدح حکم قضا کنیم <linebreak>
---------------------------------

Complete generation:
از چشم بخت خویش مبادت گزند نعمت فریب کوتاه و از می و قدح حکم قضا کنیم
